In [4]:
import gzip
import os

import pandas as pd
import numpy as np
import dask.dataframe as dd

/home/hermann/data_engineering/IMDb/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
def decompress_gz_files(input_directory, output_directory):
    # Vérifier si le répertoire de destination existe, sinon le créer
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Parcourir les fichiers dans le répertoire source
    for filename in os.listdir(input_directory):
        if filename.endswith(".gz"):
            gz_file_path = os.path.join(input_directory, filename)
            decompressed_file_path = os.path.join(output_directory, filename[:-3])  # Retirer ".gz"

            # Ouvrir le fichier .gz et décompresser son contenu dans le répertoire de destination
            with gzip.open(gz_file_path, 'rb') as gz_file:
                with open(decompressed_file_path, 'wb') as decompressed_file:
                    decompressed_file.write(gz_file.read())
            print(f"Décompressé : {filename} dans {output_directory}")

# Appel de la fonction
decompress_gz_files("../gzip", "../Data")

Décompressé : title.principals.tsv.gz dans ../Data
Décompressé : name.basics.tsv.gz dans ../Data
Décompressé : title.crew.tsv.gz dans ../Data
Décompressé : title.basics.tsv.gz dans ../Data
Décompressé : title.episode.tsv.gz dans ../Data
Décompressé : title.ratings.tsv.gz dans ../Data
Décompressé : title.akas.tsv.gz dans ../Data


In [14]:
dtype_spec = {
    'birthYear': 'object',  # Remplace par les colonnes problématiques et leurs types
    'deathYear': 'object'
}

data_name = dd.read_csv("../Data/name.basics.tsv", delimiter="\t", dtype=dtype_spec)
data_name.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [16]:
data_name.isnull().sum().compute()

nconst               0
primaryName          8
birthYear            0
deathYear            0
primaryProfession    0
knownForTitles       0
dtype: int64

In [17]:
data_name = data_name.replace(r'\N', float('nan'))

count_null_name = data_name.isnull().sum().compute()

count_null_name

nconst                      0
primaryName                58
birthYear            13213908
deathYear            13606193
primaryProfession     2677679
knownForTitles        1554685
dtype: int64